## Monomial form of polynomial interpolation
__MATH 420__ <br>
_Section 3.1_ <br>
_Spring 2021_ <br>

Let's suppose we have a bunch of points in $\mathbf{R}^2$ that have _distinct_ first coordinates. Say the points are
$$
    (x_0, y_0),  (x_1, y_1), (x_2, y_2), \dots, (x_n, y_n).
$$
The subscripts start at zero and end with $n$, so we have $n+1$ points in $\mathbf{R}^2$.

We call the first coordinates of each point, that is $x_0, x_1, \dots, x_n$ the _knots._ Since the first coordinates are distinct, the points $(x_0, y_0)$ through $(x_n, y_n)$ are points on the graph of some function. The function _isn't_ unique, but let's try to find a function in the simplest class we can think of has these points on its graph. 

The simplest largish set of functions are the polynomials, so say we want to determine a polynomial $P$ that satisfies
$$
   P(x_0) = y_0, P(x_1) = y_1, \dots, P(x_n) = y_n.
$$
This is a very famous _interpolation problem._  And the points $(x_0, y_0),  (x_1, y_1), (x_2, y_2), \dots, (x_n, y_n)$ are the _interpolation points._


Say our polynomial is
$$
   P(x) = c_0  + c_1 x + c_2 x^2 + \cdots c_m x^m.
$$
where the coefficients $c_0$ through $c_m$ are to be determined. We have $n+1$ equations and $m+1$ coefficients, so we might _guess_ that we need $m = n.$ The condition $ P(x_0) = y_0$ gives
$$
   c_0 + c_1 x_0 + c_2 x_0^2 + c_3 x_0^3 + \cdots c_n x_0^n = y_0.
$$
Although this might look like a mess, it's really a linear equation for the unknowns $c_0$ through $c_n$. Using the remaining equations, that is,  $P(x_1) = y_1, \dots, P(x_n) = y_n$ and arranging the equations in matrix form gives
$$
  \begin{bmatrix} 1 & x_0 & x_0^2 & \cdots & x_0^n \\
                 1 & x_1 & x_1^2 & \cdots & x_1^n  \\
                 \vdots & \vdots & \vdots & \vdots \\
                  1 & x_n & x_n^2 & \cdots & x_n^n
   \end{bmatrix}  
   \begin{bmatrix}
       c_0 \\
       c_1 \\
       \vdots \\
       c_n
    \end{bmatrix} =  
     \begin{bmatrix}
       y_0 \\
       y_1 \\
       \vdots \\
       y_n
    \end{bmatrix}
$$
Starting with a power of zero, each row of the coefficient matrix is an ascending list of powers of a knot. Any such matrix is a _Vandermonde_ matrix (https://en.wikipedia.org/wiki/Vandermonde_matrix). Unless the _Vandemonde_ matrix is singular (doesn't have an inverse), we've solved our problem. We'll soon prove that if the knots are distinct, a Vandermonde matrix has an inverse.


Let's look at an example.

In [ ]:
using LinearAlgebra

In [ ]:
function vandermonde(x)
    n = length(x)
    [x[i]^j for i = 1: n,j = 0 : n-1]
end

In [ ]:
M = vandermonde((0,1,2,3,4))

To solve the interpolation problem, we only need to solve a linear system.

In [ ]:
b = [1; 4; -3; 107; -46]

In [ ]:
c = M \ b


In [ ]:
P = x -> @evalpoly(x, c...)

The `(x, c...)` is a tricky Julia way to effectively do `(x, c_0, c_1, \dots, c_n)`.

Mapping $P$ onto $(0,1,2,3,4)$ should give $(1,4,-3,107, -46)$. It does

In [ ]:
map(P, (0,1,2,3,4))

Actually, _every_ time we solve a linear system, we should check the condition number:

In [ ]:
cond(M, Inf)

Compared to the machine epsilon, this isn't huge. Let's try G from GNAT. Looks like we are drinking from the fountain of success!

In [ ]:
using Gadfly

In [ ]:
plot(layer(x = [0,1,2,3,4],y=[1,4,-3,107, -46],color=[colorant"black"]),
     layer(P, -1,5),color=[colorant"purple"], Guide.title("Fourth degree polynomial interpolation"))

In [ ]:
knots = [1.0 + k/100 for k = 0 : 10]

In [ ]:
M = vandermonde(knots)

For the second coordinates of each point, let's use

In [ ]:
y = [knots[i]^2 + 1 for i = 1 : 11, j=1:1]

OK--this is tricky our points are $(x_0, 1+x_0^2), (x_1, 1+ x_1^2), \dots $


So when we solve for the coefficients $c$, we expect $c[1] = 1, c[2] = 0, c[3] = 1$ and all addition values of $c$ to vanish. But that's _not_ what we get--the values differ greatly from these values. 

In [ ]:
c = M \ y

What's the story?  The condition number for $M$ is so huge, we shouldn't expect the solution to be accurate.

In [ ]:
cond(M,Inf)

What about the residual? The residual is $M c = y$. It is

In [ ]:
M * c - y

The members of the residual are all comparable to the machine epsilon. So you might _think_ the solution is accurate, but the solution isn't accurate. Since the residual is small, graphically the solution very nearly goes through the interpolation points.

<div class="alert alert-block alert-warning"><b>Warning:</b> A small residual does not imply an accurate solution.</div>

In [ ]:
PP = x -> @evalpoly(x, c...)

In [ ]:
map(PP, knots) - y

In [ ]:
QQ = x -> x^2 + 1

In [ ]:
map(QQ, knots) - y

In [ ]:
plot(layer(PP, 1,1.1,color=[colorant"purple"]), layer(QQ,1,1.1),color=[colorant"orange"])

In [ ]:
plot(x -> PP(x)-QQ(x),1,1.1,color=[colorant"purple"])

In [ ]:
plot(layer(PP, -1,1.1), layer(QQ,-1,1.1),color=[colorant"purple"])

In [ ]:
plot(layer(x = knots, y = y,color=[colorant"black"]),
     layer(PP,1, 1.1,color=[colorant"purple"]),
     layer(QQ,1, 1.1,color=[colorant"orange"]))

In [ ]:
plot(layer(x = knots, y = y,color=[colorant"black"]),
     layer(PP,-1, 2,color=[colorant"purple"]),
     layer(QQ,-1, 2,color=[colorant"orange"]))

Let's put all this together and build a function that returns a function that interpolates given points in the $\mathbf{R}^2$. 

Do we like to be careful? Of course we do, so let's also return the matrix condition number as well as the interpolating polynomial. 

In [ ]:
function interpolating_poly(pts::Array)
    M = vandermonde( map(q -> q[1],pts))
    c = M \ map(q -> q[2],pts)
    q -> @evalpoly(q, c...), cond(M,Inf)
end    

In [ ]:
A simple test:

In [ ]:
fn = interpolating_poly([[2,3],[5,6]])

In [ ]:
fn = first(fn)

In [ ]:
fn(2)

In [ ]:
fn(5)